In [4]:
import re  
import nltk  
import string
import heapq  

C:\Users\Arpit Chandrakar\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
import nltk
nltk.data.path.append("C:/Users/Arpit Chandrakar/AppData/Roaming/nltk_data")
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Arpit
[nltk_data]     Chandrakar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Arpit
[nltk_data]     Chandrakar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import nltk
print(nltk.data.find('tokenizers/punkt'))


C:\Users\Arpit Chandrakar\AppData\Roaming\nltk_data\tokenizers\punkt


In [8]:
original_text = r"E:\CourseWork-Fall-2024\CSE-511-DPS\Project-2\graphrag\book.txt"

In [9]:
original_text = open(original_text, 'rb').read().decode(encoding = 'utf-8')

In [10]:
original_text = re.sub(r'\s+', ' ', original_text)

In [11]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
stopwords.append('to be')
stopwords.append('away')

In [13]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
def preprocessing(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)

  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = ' '.join([str(element) for element in tokens if not element.isdigit()])

  return formatted_text

In [15]:
def calculate_sentence_score(sentences, important_words, distance):
  notes = []
  indice_sentence = 0

  for sentence in [nltk.word_tokenize(sentence.lower()) for sentence in sentences]:

    indice_word = []
    for word in important_words:

      try:
        indice_word.append(sentence.index(word))
      except ValueError:
        pass
    
    indice_word.sort()


    if len(indice_word) == 0:
      continue


    list_groups = []
    group = [indice_word[0]]
    i = 1
    while i < len(indice_word):
      if indice_word[i] - indice_word[i - 1] < distance:
        group.append(indice_word[i])

      else:
        list_groups.append(group[:])
        group = [indice_word[i]]

      i += 1
    list_groups.append(group)


    high_grade_group = 0
    for g in list_groups:

      important_words_in_the_group = len(g)
      total_word_in_group = g[-1] - g[0] + 1

      note = 1.0 * important_words_in_the_group**2 / total_word_in_group


      if note > high_grade_group:
        high_grade_group = note

    notes.append((high_grade_group, indice_sentence))
    indice_sentence += 1

  return notes

In [16]:
def summarize(text, top_n_words, distance, number_sentences):
  original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
 
  formatted_sentences = [preprocessing(original_sentence) for original_sentence in original_sentences]

  words = [word for sentence in formatted_sentences for word in nltk.word_tokenize(sentence)]
 
  frequence = nltk.FreqDist(words)

  top_n_words = [word[0] for word in frequence.most_common(top_n_words)]
 
  notes_sentences = calculate_sentence_score(formatted_sentences, top_n_words, distance)
 
  best_sentences = heapq.nlargest(number_sentences, notes_sentences)

  best_sentences = [original_sentences[i] for (note, i) in best_sentences]

  return original_sentences, best_sentences, notes_sentences

In [17]:
original_sentences, best_sentences, notes_sentences = summarize(original_text, 5, 3, 3)

In [18]:
original_sentences

['\ufeffThe Project Gutenberg eBook of A Christmas Carol This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever.',
 'You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org.',
 'If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.',
 'Title: A Christmas Carol Author: Charles Dickens Illustrator: Arthur Rackham Release date: December 24, 2007 [eBook #24022] Language: English Original publication: Philadelphia and New York: J.',
 'B. Lippincott Company,, 1915 Credits: Produced by Suzanne Shell, Janet Blenkinship and the Online Distributed Proofreading Team at http://www.pgdp.net *** START OF THE PROJECT GUTENBERG EBOOK A CHRISTMAS CAROL *** Produced by Suzanne Shell, Janet Blenkinship and the Online Distributed Proo

In [24]:
import nltk
from nltk.tokenize import sent_tokenize

text = "This is a test. This is only a test."
print(sent_tokenize(text))


['This is a test.', 'This is only a test.']


In [23]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\Arpit
[nltk_data]     Chandrakar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True